In [1]:
import nltk
nltk.download('punkt')

ModuleNotFoundError: No module named 'nltk'

In [14]:
def processText(text):
    text = text.lower()
    
    text = re.sub('((www.[^s]+)|(https?://[^s]+))','',text)
    text = re.sub('@[^s]+','',text)
    text = re.sub('[s]+', ' ', text)
    text = re.sub(r'#([^s]+)', '', text)
    
    tokenized_text = word_tokenize(text)
    print(tokenized_text)
    
    output_string = ''
    
    for each in tokenized_text:
        if ( each not in string.punctuation and each not in stopwords and len(each) > 1 ):
            output_string += each + ' '


    return output_string

In [15]:
df['text'][0]
processText(df['text'][0])

['हमारे', 'गांव', 'में', 'एक', 'मास्टर', 'जी', 'रहते', 'थे', ',', 'वो', 'मोहल्ले', 'के', 'बच्चों', 'को', 'अपने', 'स्कूल', 'आने', 'को', 'कहते', 'थे।', 'जब', 'मैं', 'बड़ा', 'हुआ', 'तो', 'पता', 'चला-', 'स्कूल', 'मास्टर', 'के', 'बेटे', 'का', ',', 'बस', 'उनके', 'भतीजे', 'की', ',', 'कैंटीन', 'उनके', 'भांजे', 'की', 'और', 'पढ़ाने', 'वाले', 'भी', 'सब', 'उनके', 'रिश्तेदार', 'ही', 'थे', '..', '!', 'आज', 'मास्टर', 'जी', 'में', 'सरकार', 'में', 'मंत्री', 'हैं', '.', '!']


'हमारे गांव मास्टर जी रहते वो मोहल्ले बच्चों स्कूल आने थे। बड़ा पता चला- स्कूल मास्टर बेटे बस भतीजे कैंटीन भांजे पढ़ाने रिश्तेदार .. मास्टर जी सरकार मंत्री '

In [17]:
def tokenize_text(text):
    token_list = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if ( len (word ) < 2 ):
                continue
            token_list.append(word)
    return token_list

train_tagged = df.progress_apply(lambda r:gensim.models.doc2vec.TaggedDocument(tokenize_text(r['text']), [r['label']]), axis=1)

  0%|          | 0/16179 [00:00<?, ?it/s]

In [18]:
def doc_2_vec_model(train_tagged_):
    train_set = []
    for tag in tqdm(train_tagged):
        train_set.append(tag)

    dbow_model = gensim.models.doc2vec.Doc2Vec(vector_size=200, min_count=2, epochs=30)
    dbow_model.build_vocab([x for x in tqdm(train_set)])
    dbow_model.train(train_set, total_examples=len(train_set), epochs=50)
    return train_set, dbow_model

def vector_for_learning(model, input_docs):
    sents = input_docs
    feature_vectors, target = zip(*[(model.infer_vector(doc.words), doc.tags[0]) for doc in tqdm(sents) ])
    return feature_vectors, target

train_ds, dbow_model = doc_2_vec_model(train_tagged)
x, y = vector_for_learning(dbow_model, train_ds)
x = np.array(x)
y = np.array(y)

  0%|          | 0/16179 [00:00<?, ?it/s]

  0%|          | 0/16179 [00:00<?, ?it/s]

  0%|          | 0/16179 [00:00<?, ?it/s]

In [19]:
y = [1 if label == 'sarcastic' else 0 for label in y]
print(y)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 